# Get data using datahub catalog

datahub uri : [https://datahub.course.aiengineer.codex-platform.com/](https://datahub.course.aiengineer.codex-platform.com/)

In this notebook we want to access some data browsing the datahub catalog.

We should be able to see our chicago data in clickhouse, thanks to the [../Data_Platform_Foundation/1_empower_analysis_with_db_and_viz.ipynb](../Data_Platform_Foundation/1_empower_analysis_with_db_and_viz.ipynb) notebook

Let's verify that the data is in clickhouse before continuing

## 0.0 Prerequisite : chigago taxi data in clickhouse

In [ ]:
#pip install pandahouse minio
import pandahouse as ph
import pandas as pd

In [ ]:
### helper function for handle this python client
def write_clickhouse(query,connection):
    print(query)
    try:
        ph.read_clickhouse(query,connection=connection)
    except KeyError:
        print("Nothing to return")

In [ ]:
##db name is firstname_lastname like your username but with "_" instead of "-"
dbname = ''#firstname_lastname
dbtable='chicago_taxi'
## connect with your dbname
connection = dict(database=dbname,
                  host='http://clickhouse-install.clickhouse.svc.cluster.local:8123',
                  user='admin',
                  password='B1gdata-demo')

In [ ]:
# verifiy if the db exists
ph.read_clickhouse(f"show databases",connection=connection)

If you see your database, verify your dataset is present, if you can't see your database : go to 0.1, otherwise, go directly to 1.0

In [ ]:
ph.read_clickhouse(f"select * from {dbname}.{dbtable} limit 5",connection=connection).head()

## 0.1 Recover taxi trips and push into clickhouse (if it is not already here)

In [ ]:
# Get the dataset Taxi Trips as CSV
!curl --get 'https://data.cityofchicago.org/resource/wrvz-psew.csv' \
  --data-urlencode '$limit=10000' \
  --data-urlencode '$where=trip_start_timestamp >= "2023-01-01" AND trip_start_timestamp < "2023-02-01"' \
  --data-urlencode '$select=tips,trip_start_timestamp,trip_seconds,trip_miles,pickup_community_area,pickup_centroid_latitude,pickup_centroid_longitude,dropoff_community_area,fare,tolls,extras,trip_total' \
  | tr -d '"' > "./chicagodata/trip.csv"

In [ ]:
#pip install pandahouse minio
import pandahouse as ph
import pandas as pd

In [ ]:
## create your db name with your username but with "_" instead of "-"
dbname = ''#firstname_lastname

## The connection dict need a default database
connection = dict(database='default',
                  host='http://clickhouse-install.clickhouse.svc.cluster.local:8123',
                  user='admin',
                  password='B1gdata-demo')


write_clickhouse(f"create database {dbname}",connection)

connection['database'] = f"{dbname}"

print(connection)

In [ ]:
## get data
dbtable='chicago_taxi'
data = pd.read_csv("./chicagodata/trip.csv")
### select features
features = data[[
    "tips",
    "trip_start_timestamp",
    "trip_seconds",
    "trip_miles",
    "pickup_community_area" ,
    "dropoff_community_area" ,
    "fare",
    "tolls",
    "extras",
    "trip_total"
]]

In [ ]:
### create table for inserting taxi trip dataset 
## Clickhouse table definition
# using the df informations, and clickhouse documentation write  the create table statement
taxitable = f"""
CREATE TABLE {dbname}.{dbtable}
(
    `tips` Float32,
    `trip_start_timestamp` DateTime,
    `trip_seconds` Float32,
    `trip_miles` Float32,
    `pickup_community_area` Float32,
    `dropoff_community_area` Float32,
    `fare` Float32,
    `tolls` Float32,
    `extras` Float32,
    `trip_total` Float32
) 
ENGINE = MergeTree
PARTITION BY toYYYYMM(trip_start_timestamp)
ORDER BY trip_start_timestamp;
"""

In [ ]:
write_clickhouse(taxitable,connection)


In [ ]:
## We have to be compliant with the clickhouse date type. 
## we need to force '%Y-%m-%d %H:%M:%S'
## force the date format with the defined schema, using pandas
features["trip_start_timestamp"] = pd.to_datetime(data["trip_start_timestamp"]).dt.strftime('%Y-%m-%d %H:%M:%S')

In [ ]:
### insert using the to_clickhouse function
ph.to_clickhouse(features, dbtable, index=False, chunksize=100000, connection=connection)

---

## 1.0 Browse UI to see our taxi trips dataset

![datahub](./images/datahub.png)

## 1.1 Create a transformation view on the data  

Here we want to create a view with only 1 week of data, and see the lineage on the catalog tool

In [ ]:
dbview='chicago_data_oneweek'

In [ ]:
### Create a view from chicago_taxi table
# this view will use only last week of available data
taxiview = f"""
CREATE view {dbname}.{dbview} as Select * from {dbname}.{dbtable}  where trip_start_timestamp >  (toDateTime('2023-02-01') - INTERVAL 7 DAY)
"""

In [ ]:
write_clickhouse(taxiview,connection)

## 1.2 Create another view only on geo data

In [ ]:
dbview='chicago_data_geo'

In [ ]:
### Create a view from chicago_taxi table
# this view will use only last week of available data
geoview = f"""
CREATE view {dbname}.{dbview} as Select pickup_community_area, dropoff_community_area from {dbname}.{dbtable}
"""

In [ ]:
write_clickhouse(geoview,connection)

### 1.3 Browse UI to get the lineage link between the table and the view

![lineage](./images/lineage.PNG)